In [ ]:
import pandas as pd
import math
import tkinter as tk
from tkinter import ttk

# Cargar todas las hojas en un diccionario de DataFrames
excel_tablas = "Tablas PER 2000 y 2020.xlsx"
hojas = pd.read_excel(excel_tablas, sheet_name=None, header=1)
xlsx = pd.ExcelFile(excel_tablas)
nombres_pestañas = xlsx.sheet_names

# Función para leer tablas
def leer_tablas(nombre_tabla):
    # Crear una copia del DataFrame para no modificar el original
    df = hojas[nombre_tabla].copy()
    print(f"Columnas originales de la tabla {nombre_tabla}: {df.columns.tolist()}")
    print(f"Número de columnas: {len(df.columns)}")
    print(f"Primeras 5 filas de la tabla {nombre_tabla} (antes de procesar):")
    print(df.head())

    # Asegurarnos de que el índice sea 'x+t' o similar
    if df.index.name is None or 'x+t' not in df.index.name.lower():
        # Buscar una columna que contenga 'x+t' o similar
        index_col = None
        for col in df.columns:
            if 'x+t' in col.lower():
                index_col = col
                break
        if index_col:
            df.set_index(index_col, inplace=True)
        else:
            raise ValueError(f"No se encontró una columna de índice 'x+t' en la tabla {nombre_tabla}.")

    # Renombrar columnas
    print(f"Columnas después de establecer el índice: {df.columns.tolist()}")
    print(f"Número de columnas después de establecer el índice: {len(df.columns)}")
    if len(df.columns) == 2:
        df.columns = ['qx', 'mejora']
    elif len(df.columns) == 1:
        df.columns = ['qx']
    else:
        raise ValueError(f"La tabla {nombre_tabla} tiene un número inesperado de columnas: {len(df.columns)}")

    df.index.name = 'edad'
    print(f"Primeras 5 filas de la tabla {nombre_tabla} (después de renombrar):")
    print(df.head())
    # Validar que los índices sean numéricos
    if not pd.api.types.is_numeric_dtype(df.index):
        raise ValueError(f"El índice de la tabla {nombre_tabla} no es numérico.")
    return df

# Función para determinar el año base según la tabla
def generacion(nombre_tabla, g):
    if nombre_tabla in ['PERM2000C', 'PERF2000C', 'PERM2000P', 'PERF2000P']:
        anio_base = 2000
        if g > 2000:
            raise ValueError("La generación no puede ser mayor a 2000.")
    elif nombre_tabla in ['PERM_2020_Indiv_2Orden', 'PERM_2020_Indiv_1Orden',
                          'PERF_2020_Indiv_2Orden', 'PERF_2020_Indiv_1Orden',
                          'PERM_2020_Colectivos_2Orden', 'PERM_2020_Colectivos_1Orden',
                          'PERF_2020_Colectivos_2Orden', 'PERF_2020_Colectivos_1Orden']:
        anio_base = 2012
        if g > 2012:
            raise ValueError("La generación no puede ser mayor a 2012.")
    else:
        raise ValueError("Tabla no reconocida para cálculo generacional")
    return anio_base

# Función de ajuste de qx
def q_x(qx, mejora, t):
    if mejora is None:
        return qx
    qx_ajustado = qx * math.exp(-mejora * t)
    # Forzar que qx_ajustado esté entre 0 y 1
    if qx_ajustado < 0:
        print(f"Advertencia: qx_ajustado={qx_ajustado} es menor que 0 para qx={qx}, mejora={mejora}, t={t}. Ajustando a 0.")
        return 0
    if qx_ajustado > 1:
        print(f"Advertencia: qx_ajustado={qx_ajustado} es mayor que 1 para qx={qx}, mejora={mejora}, t={t}. Ajustando a 1.")
        return 1
    return qx_ajustado

# Generar tabla generacional
def generar_tabla_generacional(g, nombre_tabla, edad_actual, anio_evaluacion):
    anio_base = generacion(nombre_tabla, g)
    t_inicio = anio_evaluacion - anio_base
    tabla_base = leer_tablas(nombre_tabla)

    max_edad = int(tabla_base.index.max())
    x_mas_t_list = list(range(edad_actual, max_edad + 1))
    edad_t = list(range(0, len(x_mas_t_list)))

    resultados = pd.DataFrame(index=edad_t, columns=['x+t', 'qx+t ajustado', 'lx', 'dx'])
    resultados['x+t'] = x_mas_t_list
    resultados['lx'] = 0.0  # Inicializar como float64

    num_columnas = len(tabla_base.columns)
    for i, x in enumerate(x_mas_t_list):
        if x <= max_edad:
            try:
                qx = tabla_base.loc[x, 'qx']
                mejora = tabla_base.loc[x, 'mejora'] if num_columnas > 1 else None
                t = t_inicio + (x - edad_actual)
                qx_aj = q_x(qx, mejora, t)
                resultados.loc[i, 'qx+t ajustado'] = qx_aj
                if x <= edad_actual + 5:  # Imprimir los primeros 5 para depurar
                    print(f"x={x}, qx_base={qx}, mejora={mejora}, t={t}, qx_ajustado={qx_aj}")
            except KeyError as e:
                print(f"Error: No se encontró la edad {x} en la tabla {nombre_tabla}. Error: {e}")
                resultados.loc[i, 'qx+t ajustado'] = 1.0
        else:
            resultados.loc[i, 'qx+t ajustado'] = 1.0

    resultados.loc[0, 'lx'] = 1000000.0
    for i in range(1, len(resultados)):
        qx_prev = resultados.loc[i-1, 'qx+t ajustado']
        resultados.loc[i, 'lx'] = resultados.loc[i-1, 'lx'] * (1 - qx_prev)
        if i <= 5:  # Imprimir los primeros 5 para depurar
            print(f"i={i}, qx_prev={qx_prev}, lx_{i}={resultados.loc[i, 'lx']}")

    resultados['dx'] = resultados['lx'] * resultados['qx+t ajustado']
    print("Tabla generacional (primeras 10 filas):")
    print(resultados.head(10))
    return resultados

# Función para calcular VPA
def calcular_vpa(tabla, edad_actual, tasa, tipo_renta, n=None):
    v = 1 / (1 + tasa)
    ax = 0
    lx0 = tabla.loc[0, 'lx']
    print(f"v = {v}, lx0 = {lx0}")

    if tipo_renta == "vitalicia_pospagable":
        for t in range(1, len(tabla)):
            tpx = tabla.loc[t, 'lx'] / lx0
            term = (v ** t) * tpx
            ax += term
            if t <= 5:
                print(f"t={t}, lx_{t}={tabla.loc[t, 'lx']:.2f}, tpx={tpx:.6f}, v^{t}={v**t:.6f}, término={term:.6f}")

    elif tipo_renta == "vitalicia_prepagable":
        for t in range(0, len(tabla)):
            tpx = tabla.loc[t, 'lx'] / lx0
            ax += (v ** t) * tpx

    elif tipo_renta == "temporal_pospagable":
        if n is None:
            raise ValueError("Debe especificar duración n para renta temporal.")
        for t in range(1, min(n + 1, len(tabla))):
            tpx = tabla.loc[t, 'lx'] / lx0
            ax += (v ** t) * tpx

    elif tipo_renta == "temporal_prepagable":
        if n is None:
            raise ValueError("Debe especificar duración n para renta temporal.")
        for t in range(0, min(n, len(tabla))):
            tpx = tabla.loc[t, 'lx'] / lx0
            ax += (v ** t) * tpx

    else:
        raise ValueError("Tipo de renta no reconocido.")

    return ax

# Función principal de cálculo
def calcular():
    try:
        # Validar campos vacíos
        if not entry_anio_nacimiento.get():
            raise ValueError("El campo 'Año de nacimiento' no puede estar vacío.")
        if not entry_edad.get():
            raise ValueError("El campo 'Edad actual' no puede estar vacío.")
        if not entry_tasa.get():
            raise ValueError("El campo 'Tasa de interés (%)' no puede estar vacío.")
        if not entry_renta.get():
            raise ValueError("El campo 'Monto de renta' no puede estar vacío.")

        g = int(entry_anio_nacimiento.get())
        nombre_tabla = combo_tabla.get()
        if not nombre_tabla:
            raise ValueError("Debe seleccionar una tabla de mortalidad.")
        edad_actual = int(entry_edad.get())
        tasa = float(entry_tasa.get()) / 100
        renta = float(entry_renta.get())
        tipo_renta = combo_tipo_renta.get()
        if not tipo_renta:
            raise ValueError("Debe seleccionar un tipo de renta.")
        n = int(entry_duracion.get()) if tipo_renta.startswith("temporal") and entry_duracion.get() else None
        anio_evaluacion = g + edad_actual

        print(f"Datos ingresados: g={g}, tabla={nombre_tabla}, edad={edad_actual}, tasa={tasa}, renta={renta}, tipo={tipo_renta}, n={n}, año_eval={anio_evaluacion}")

        tabla_generacional = generar_tabla_generacional(g, nombre_tabla, edad_actual, anio_evaluacion)
        ax = calcular_vpa(tabla_generacional, edad_actual, tasa, tipo_renta, n)
        prima = renta * ax
        resultado.config(text=f"a_{edad_actual}: {ax:.2f}\nPrima única: {prima:.2f}")
        print(f"Resultado: a_{edad_actual}={ax:.2f}, Prima={prima:.2f}")
    except Exception as e:
        resultado.config(text=f"Error: {str(e)}")
        print(f"Error: {str(e)}")

# Interfaz gráfica
root = tk.Tk()
root.title("Calculadora Actuarial")

tk.Label(root, text="Año de nacimiento:").grid(row=0, column=0, padx=5, pady=5)
entry_anio_nacimiento = tk.Entry(root)
entry_anio_nacimiento.grid(row=0, column=1)

tk.Label(root, text="Tabla de mortalidad:").grid(row=1, column=0, padx=5, pady=5)
combo_tabla = ttk.Combobox(root, values=['PERM2000C', 'PERF2000C', 'PERM2000P', 'PERF2000P',
                                         'PERM_2020_Indiv_2Orden', 'PERM_2020_Indiv_1Orden',
                                         'PERF_2020_Indiv_2Orden', 'PERF_2020_Indiv_1Orden',
                                         'PERM_2020_Colectivos_2Orden', 'PERM_2020_Colectivos_1Orden',
                                         'PERF_2020_Colectivos_2Orden', 'PERF_2020_Colectivos_1Orden'])
combo_tabla.grid(row=1, column=1)

tk.Label(root, text="Edad actual:").grid(row=2, column=0, padx=5, pady=5)
entry_edad = tk.Entry(root)
entry_edad.grid(row=2, column=1)

tk.Label(root, text="Tasa de interés (%):").grid(row=3, column=0, padx=5, pady=5)
entry_tasa = tk.Entry(root)
entry_tasa.grid(row=3, column=1)

tk.Label(root, text="Monto de renta:").grid(row=4, column=0, padx=5, pady=5)
entry_renta = tk.Entry(root)
entry_renta.grid(row=4, column=1)

tk.Label(root, text="Duración (si temporal):").grid(row=5, column=0, padx=5, pady=5)
entry_duracion = tk.Entry(root)
entry_duracion.grid(row=5, column=1)

tk.Label(root, text="Tipo de renta:").grid(row=6, column=0, padx=5, pady=5)
combo_tipo_renta = ttk.Combobox(root, values=["vitalicia_pospagable", "vitalicia_prepagable",
                                               "temporal_pospagable", "temporal_prepagable"])
combo_tipo_renta.grid(row=6, column=1)

ttk.Button(root, text="Calcular", command=calcular).grid(row=7, column=0, columnspan=2, pady=10)
resultado = tk.Label(root, text="")
resultado.grid(row=8, column=0, columnspan=2)

root.mainloop()

Datos ingresados: g=1955, tabla=PERF2000P, edad=50, tasa=0.03, renta=20000.0, tipo=vitalicia_pospagable, n=None, año_eval=2005
Columnas originales de la tabla PERF2000P: ['x+t', 'qx+t, tabla base', 'λx+t']
Número de columnas: 3
Primeras 5 filas de la tabla PERF2000P (antes de procesar):
   x+t  qx+t, tabla base  λx+t
0    0          0.003215  0.04
1    1          0.000264  0.04
2    2          0.000196  0.04
3    3          0.000132  0.04
4    4          0.000128  0.04
Columnas después de establecer el índice: ['qx+t, tabla base', 'λx+t']
Número de columnas después de establecer el índice: 2
Primeras 5 filas de la tabla PERF2000P (después de renombrar):
            qx  mejora
edad                  
0     0.003215    0.04
1     0.000264    0.04
2     0.000196    0.04
3     0.000132    0.04
4     0.000128    0.04
x=50, qx_base=0.00122, mejora=0.025, t=5, qx_ajustado=0.0010766462211532063
x=51, qx_base=0.001385, mejora=0.025, t=6, qx_ajustado=0.001192080547348705
x=52, qx_base=0.001449, m